In [1]:
# to handle data
import numpy as np
import pandas as pd

# financial data APIs
import pandas_datareader.data as web
import yfinance as yf
import gspread as gs

# to plot the graphs
import matplotlib.pyplot as plt
import seaborn as sns

# to optimize
import scipy.optimize as sco

# to CML plot
import scipy.interpolate as sci

from datetime import datetime

# Input

## Import customer's choice
### Connect to Figma & Google sheets

### Identify risk tolerance level
Question 1,2,5,6 are used

In [2]:
score_dict = [
    # question 1
    {'20s (including 18-19 years old)': 5,
     '30s': 4,
     '40s': 3,
     '50s': 2,
     'Over 60s': 1},
    
    # question 2
    {'$12,000 - $24,000': 1,
     '$24,000 - $48,000': 2,
     '$48,000 - $84,000': 3,
     'More than $84,000': 4},
    
    # question 5
    {'Less than 10%': 1,
     '10-20%': 2,
     '20-30%': 3,
     '30-40%': 4,
     'More than 40%': 5},
    
    # question 6
    {'I would be worried and consider moving to safer investments.': 1,
     'I might consider making some adjustments to my portfolio.': 2,
     "I'd be patient and confident that my investments will rebound.": 3,
     "I wouldn't be concerned; I understand that markets have ups and downs.": 4}
]

In [3]:
# collect customer's choices from Figma
q1 = 1
q2 = 1
q5 = 1
q6 = 1

# get score
q1 = score_dict.get(q1)
q2 = score_dict.get(q2)
q5 = score_dict.get(q5)
q6 = score_dict.get(q6)

AttributeError: 'list' object has no attribute 'get'

In [ ]:
# calculate score of risk tolerance
total_score = q1+q2+q5+q6

# classify
if total_score > 20:
    level = 5
elif total_score > 15:
    level = 4
elif total_score > 10:
    level = 3
elif total_score > 5:
    level = 2
else:
    level = 1

### Collect customer's choice of stocks

In [ ]:
# get from Figma
stock_list = 'TSLA, AMZN,AAPL'

# convert to list and remove whitespace
stock_list = [stock.strip() for stock in stock_list.split(',')]

## Import historical data
* What are the assets?
    * Random 10 stock in S&P500 (chọn bừa để thử chạy các phần sau thôi)
* What is the period?
    * 2018-01-01 to today

In [ ]:
# Get the list of S&P 500 tickers
sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500_tickers = sp500_tickers['Symbol'].tolist()

# Define the start and end dates for historical data
start_date = '2015-01-01'
end_date = datetime.today().strftime('%Y-%m-%d')

# Loop through each ticker and collect historical data
stocks = pd.DataFrame()

for ticker in sp500_tickers:
    try:
        # Download historical data for the current ticker
        data = yf.download(ticker, start=start_date, end=end_date)
        
        # Add the 'Close' prices to the stocks DataFrame - ONLY STOCK WITH HIGHER RETURN OF MORE THAN 3%
        if np.log(data['Close'] / data['Close'].shift(1)).mean()*252 >= 0.05:
            print(np.log(data['Close'] / data['Close'].shift(1)).mean()*252)
            stocks[ticker] = data['Close']
        
        print(f"Downloaded data for {ticker}")
    except Exception as e:
        print(f"Error downloading data for {ticker}: {str(e)}")
        

noa = len(stocks.columns)


[*********************100%%**********************]  1 of 1 completed
Downloaded data for MMM
[*********************100%%**********************]  1 of 1 completed
0.10471014097373178
Downloaded data for AOS
[*********************100%%**********************]  1 of 1 completed
0.08456558886642712
Downloaded data for ABT
[*********************100%%**********************]  1 of 1 completed
0.08701372543959232
Downloaded data for ABBV
[*********************100%%**********************]  1 of 1 completed
0.14329472283176775
Downloaded data for ACN
[*********************100%%**********************]  1 of 1 completed
Downloaded data for ADM
[*********************100%%**********************]  1 of 1 completed
0.23656276675101306
Downloaded data for ADBE
[*********************100%%**********************]  1 of 1 completed
0.1110604136069105
Downloaded data for ADP
[*********************100%%**********************]  1 of 1 completed
Downloaded data for AES
[*********************100%%***************

KeyboardInterrupt: 

# Data preprocessing
* Detect and remove missing values
* Outliers should not be considered here, because the data is a time-series

In [ ]:
stocks.describe()

,AOS,ABT,ABBV,ACN,ADBE,ADP,AFL,A
count,2228.000000,2228.000000,2228.000000,2228.000000,2228.000000,2228.000000,2228.000000,2228.000000
mean,53.638842,78.911683,96.269214,198.170009,292.207128,152.216050,46.933817,86.302724
std,12.366540,29.670182,32.188755,81.766556,167.285148,54.157598,12.789365,38.643246
min,27.115000,36.340000,48.270000,84.029999,69.989998,73.519997,25.270000,33.369999
25%,45.237501,48.437499,66.692497,121.100000,126.284998,101.930000,36.020000,52.697499
50%,53.535000,78.940002,90.290001,177.400002,275.544998,146.365005,45.135000,73.570000
75%,63.300001,106.975000,116.052502,275.475006,433.422508,203.145000,55.352499,122.332499
max,85.849998,141.460007,174.960007,415.420013,688.369995,269.899994,82.230003,179.279999


In [ ]:
# detect missing values
stocks.isna().sum()

TSLA    0
AMZN    0
AAPL    0
dtype: int64

There is no missing values -> Next step

# Recommended portfolio
## Bootstrap
2000 samples of weights

In [ ]:
# calculate return of each asset
log_return = np.log(stocks / stocks.shift(1))

# define function of portolio return
def p_return(weights):
  return np.sum(log_return.mean()*weights)*252

# define function of portolio risk
def p_std(weights):
  return np.sqrt(np.dot(weights.T, np.dot(log_return.cov()*252, weights)))

In [ ]:
log_return.mean()

TSLA    0.001591
AMZN    0.000595
AAPL    0.000978
dtype: float64

In [ ]:
p_return_2000 = []
p_std_2000 = []
rand_weight = []

for _ in range(2000): 
  # create a sample of weights (2000)
  weights = np.random.random(noa)
  weights /= np.sum(weights)
  
  # calculate portfolio return and risk
  p_return_2000.append(p_return(weights))
  p_std_2000.append(p_std(weights))
  rand_weight.append(weights)

# convert from list to array
p_return_2000 = np.array(p_return_2000)
p_std_2000 = np.array(p_std_2000)

## Efficient Frontier
The idea is that EF is formed by portfolios with minimum risk (min var) for each return. 

To be clearer, each return has many many portfolios whose weights can generate that return - these are basically on the horizontal line for each return. And of course, the "efficient: portfolio is the portfolio whose minimum variance among those portfolios

`scipy.opmize,minimize` is used to find those efficient portfolios
* Method: Sequential Least Quadratic Programming
* Constraint: 
    * $\Sigma weight = 1$
    * $R_p = R_{target} $
* Boundary of each asset: $weight_i \in [0,1]$
* Minimize variance

### Set constraints

In [ ]:
# constraints
## the meaning of the first line is: Rp-Rtarget = 0 (type:eq means "=0"). Similar to the second line
constraints = ({"type":"eq", "fun": lambda x: p_return(x)-target},
               {"type":"eq", "fun": lambda x: np.sum(x)-1})

# set boundary
boundaries = tuple((0,1) for _ in range(noa))

### Find a collection of "efficient" portfolios

In [ ]:
target_return = np.linspace(min(p_return_2000),max(p_return_2000),100) # there are 100 return targets
target_std = []
pweights = []
x0 = np.array(noa*[1./noa])

# find efficient portfolio for each of target return
## x0 assigned as follow means the first weight data entry is every asset has the same weight
for target in target_return:
  res = sco.minimize(p_std, x0=x0, method="SLSQP", constraints=constraints, bounds=boundaries)
  target_std.append(res["fun"])
  pweights.append(res["x"])

target_std = np.array(target_std)

## Optimal Risky portfolio by Risk Tolerance Level
The idea here is that pick 5 portfolios from 100 "efficient" portfolios above *(see p2)*:
* **Level 1 - Risk averse**: minimum risk portfolio *(not necessarily be MVP - because MVP is generated by `scipy.optimize.minimize`, while 2000 samples are generated randomly)*
* **Level 2**: median of level 1 and 3
* **Level 3 - Risk neutral**: maximum $\frac{R_p}{\sigma_p}$ *(in other words, best trade-off)*
* **Level 4**: median of level 3 and 5
* **Level 5 - Risk lover**: maximum return

In [ ]:
# find the position of each level in 100 "efficient" portfolios
lv1_index = np.argmin(target_std)
lv3_index = np.argmax(target_return/target_std)
lv5_index = np.argmax(target_return)
lv2_index = int((lv1_index+lv3_index)/2)
lv4_index = int((lv3_index+lv5_index)/2)
lv_index = {k:v for k,v in zip(range(1,6),[lv1_index, lv2_index, lv3_index, lv4_index, lv5_index])} # summary


In [ ]:
# define a function to return return, risk, weight for each level
def optimal_portfolio(level, stock_df):
    # find the weight
    optimal_weight = pweights[lv_index[level]]
    
    # print out weight for each stock
    for stock,weight in zip(stock_df.columns, optimal_weight):
        print(stock,':',round(weight*100, 3),'%')
    
    # print out risk and return of the portfolio
    print(f'Portfolio return (lv{level}): {round(p_return(optimal_weight)*100,3)}%')
    print(f'Portfolio std (lv{level}): {round(p_std(optimal_weight),3)}')
    
    return optimal_weight

In [ ]:
w = optimal_portfolio(level,stocks)

TSLA : 0.0 %
AMZN : 36.824 %
AAPL : 63.176 %
Portfolio return (lv1): 21.098%
Portfolio std (lv1): 0.302


In [ ]:
w

array([2.77555756e-17, 3.68242816e-01, 6.31757184e-01])

In [ ]:
import plotly.express as px